# Pythran version of the Julia challenge

Pythran compiles Numpy code into native code, with an emphasis on numerical kernels. In some way it does the same as Julia, but for Numerical Python: use a single language to catch'em all.

## Requirements

    pip install numpy
    pip install pythran colorlog

In [1]:
import pythran
%load_ext pythran.magic

WARNING  Pythran support disabled for module: scipy.special
WARNING  Pythran support disabled for module: scipy


In [2]:
import numpy as np
a = np.random.random((1000,1000))
b = np.random.random(1000)
c = np.random.rand()

In [3]:
ref = a + b + np.sin(c)

The magic below calls Pythran to provide an accelerated, vectorized version, loop-merged kernel. Pythran can support much more complex code than that, obvisouly :-)

In [4]:
%%pythran -DUSE_BOOST_SIMD -march=native -O3
import numpy as np
#pythran export kernel(float64[:,:], float64[:], float64)
def kernel(a, b, c):
    return a + b + np.sin(c)

Checking that the result is correct

In [5]:
out = kernel(a, b, c)

In [6]:
np.all(ref == out)

True

And benchmarking with %timeit, compared to numpy.

In [7]:
%timeit a + b + np.sin(c)

1.71 ms ± 21 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
%timeit kernel(a, b, c)

914 µs ± 8.54 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Bonus: alternatives

Naively exploring other straight-forward alternatives.

### requirements

    pip install numba numexpr cython

In [9]:
import numexpr as ne

In [10]:
%timeit ne.evaluate("a + b * sin(c)")

4.35 ms ± 23.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
import numba
@numba.jit(nopython=True)
def kernel(a, b, c):
    return a + b + np.sin(c)

In [12]:
%timeit kernel(a,b,c)

1.12 ms ± 19.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%load_ext cython

In [14]:
%%cython --compile-args=-O3 --compile-args=-march=native

import numpy as np
cimport numpy as np
cimport cython

cdef extern from "math.h":
    double sin(double)


@cython.boundscheck(False)
def kernel(np.ndarray[np.float64_t, ndim=2] a,
                np.ndarray[np.float64_t, ndim=1] b,
                np.float64_t c):

    # Local variables
    cdef Py_ssize_t i, j
    cdef np.ndarray[np.float64_t, ndim=2] out

    out = np.empty_like(a)

    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            out[i,j] = a[i,j] + b[i] + sin(c)

    return out

In [15]:
%timeit kernel(a,b,c)

1.21 ms ± 15.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
